### Machine Learning
#### MAP@K for ```K=7```

* Gonzalo Fuertes Sémelas
* Álvaro Pérez Trasancos
* Miguel Chacón Maldonado
* Ubaldo Peralta Sánchez
* Álvaro Nicholas García Crawford
* Miguel López Melgar

Load the predictions attached in the original handing, as well as the solutions:

In [1]:
import pandas as pd

# Load CSV with the predictions ("preds") and CSV with the solutions ("actual")
preds = pd.read_csv('predictions.csv')
actual = pd.read_csv('../soluciones.csv')

# List of products
targets = [col for col in preds.columns if 'ind_prod' in col]

Create Series with ```cod_persona``` as index and a list of the contracted products as values, both for ```preds``` and ```actual```. Then merge both Series in a dataframe and drop the rows with empty lists in the solutions.

In [2]:
def reverse_onehot(row):
    prods = []
    for target in targets:
        if row[target] == 1:
            prods.append(target)
    return prods

# Create Series with the predictions (with 'cod_persona' as index)
preds_byclient = preds.apply(reverse_onehot, axis=1)
preds_byclient.index = preds['cod_persona']

# Create Series with the solutions (with 'cod_persona' as index)
actual_byclient = actual.apply(reverse_onehot, axis=1)
actual_byclient.index = actual['cod_persona']

# Join Series by 'cod_persona'
ps = actual_byclient.to_frame('Solutions').merge(preds_byclient.to_frame('Predictions'), left_index=True, right_index=True)

The evaluation is going to be made through the metric MAP@K. Here this metric is defined:

In [3]:
def apk(actual, predicted, k):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0
    num_hits = 0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1
            score += num_hits / (i + 1)
    if not actual:
        return 0
    return score / min(len(actual), k)

def mapk(actual, predicted, k):
    from numpy import mean
    return mean([apk(a, p, k) for a, p in zip(actual, predicted)])

Metric applied for ```K=7``` and normalized over the solutions with themselves:

In [4]:
metric_actual = mapk(ps['Solutions'], ps['Solutions'], 7)
metric_preds = mapk(ps['Solutions'], ps['Predictions'], 7)

metric_results = metric_preds / metric_actual
metric_results

0.20050257129499763